# Mekanisme Perhatian dan Transformer

Salah satu kelemahan utama jaringan berulang (recurrent networks) adalah semua kata dalam sebuah urutan memiliki dampak yang sama terhadap hasil. Hal ini menyebabkan performa yang kurang optimal pada model encoder-decoder LSTM standar untuk tugas sequence-to-sequence, seperti Named Entity Recognition dan Machine Translation. Pada kenyataannya, kata-kata tertentu dalam urutan input sering kali memiliki pengaruh lebih besar terhadap output berurutan dibandingkan yang lain.

Pertimbangkan model sequence-to-sequence, seperti penerjemahan mesin. Model ini diimplementasikan dengan dua jaringan berulang, di mana satu jaringan (**encoder**) akan merangkum urutan input menjadi state tersembunyi, dan jaringan lainnya, **decoder**, akan mengurai state tersembunyi ini menjadi hasil terjemahan. Masalah dengan pendekatan ini adalah state akhir dari jaringan akan kesulitan mengingat awal kalimat, sehingga menyebabkan kualitas model yang buruk pada kalimat panjang.

**Mekanisme Perhatian** menyediakan cara untuk memberikan bobot pada dampak kontekstual dari setiap vektor input terhadap setiap prediksi output dari RNN. Cara ini diimplementasikan dengan menciptakan jalur pintas antara state antara dari RNN input dan RNN output. Dengan cara ini, saat menghasilkan simbol output $y_t$, kita akan mempertimbangkan semua state tersembunyi input $h_i$, dengan koefisien bobot yang berbeda $\alpha_{t,i}$. 

![Gambar menunjukkan model encoder/decoder dengan lapisan perhatian aditif](../../../../../translated_images/id/encoder-decoder-attention.7a726296894fb567.webp)
*Model encoder-decoder dengan mekanisme perhatian aditif dalam [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), dikutip dari [blog ini](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Matriks perhatian $\{\alpha_{i,j}\}$ akan merepresentasikan sejauh mana kata-kata tertentu dalam input berperan dalam menghasilkan kata tertentu dalam urutan output. Di bawah ini adalah contoh matriks seperti itu:

![Gambar menunjukkan contoh alignment yang ditemukan oleh RNNsearch-50, diambil dari Bahdanau - arviz.org](../../../../../translated_images/id/bahdanau-fig3.09ba2d37f202a6af.webp)

*Gambar diambil dari [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Gambar 3)*

Mekanisme perhatian bertanggung jawab atas banyak pencapaian terkini atau mendekati terkini dalam pemrosesan bahasa alami. Namun, menambahkan perhatian secara signifikan meningkatkan jumlah parameter model, yang menyebabkan masalah skala pada RNN. Salah satu kendala utama dalam menskalakan RNN adalah sifat berulang dari model ini yang membuat pelatihan sulit untuk dibatch dan diparalelkan. Dalam RNN, setiap elemen dari urutan harus diproses secara berurutan, yang berarti tidak dapat dengan mudah diparalelkan.

Adopsi mekanisme perhatian yang dikombinasikan dengan kendala ini mengarah pada terciptanya Model Transformer yang kini menjadi State of the Art, seperti yang kita kenal dan gunakan saat ini, mulai dari BERT hingga OpenGPT3.

## Model Transformer

Alih-alih meneruskan konteks dari setiap prediksi sebelumnya ke langkah evaluasi berikutnya, **model transformer** menggunakan **positional encoding** dan **perhatian** untuk menangkap konteks dari input tertentu dalam jendela teks yang diberikan. Gambar di bawah ini menunjukkan bagaimana positional encoding dengan perhatian dapat menangkap konteks dalam jendela tertentu.

![GIF animasi menunjukkan bagaimana evaluasi dilakukan dalam model transformer.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

Karena setiap posisi input dipetakan secara independen ke setiap posisi output, transformer dapat melakukan paralelisasi lebih baik dibandingkan RNN, yang memungkinkan model bahasa yang jauh lebih besar dan lebih ekspresif. Setiap kepala perhatian dapat digunakan untuk mempelajari hubungan yang berbeda antara kata-kata, yang meningkatkan tugas Pemrosesan Bahasa Alami di hilir.

## Membangun Model Transformer Sederhana

Keras tidak memiliki lapisan Transformer bawaan, tetapi kita dapat membangunnya sendiri. Seperti sebelumnya, kita akan fokus pada klasifikasi teks dari dataset AG News, tetapi perlu dicatat bahwa model Transformer menunjukkan hasil terbaik pada tugas NLP yang lebih sulit.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Lapisan baru dalam Keras harus merupakan subclass dari kelas `Layer`, dan mengimplementasikan metode `call`. Mari kita mulai dengan lapisan **Positional Embedding**. Kita akan menggunakan [beberapa kode dari dokumentasi resmi Keras](https://keras.io/examples/nlp/text_classification_with_transformer/). Kita akan mengasumsikan bahwa kita menyisipkan semua urutan input hingga panjang `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Lapisan ini terdiri dari dua lapisan `Embedding`: untuk menyematkan token (dengan cara yang telah kita bahas sebelumnya) dan posisi token. Posisi token dibuat sebagai urutan bilangan alami dari 0 hingga `maxlen` menggunakan `tf.range`, lalu diteruskan melalui lapisan embedding. Dua vektor embedding yang dihasilkan kemudian dijumlahkan, menghasilkan representasi input yang disematkan secara posisi dengan bentuk `maxlen`$\times$`embed_dim`.

Sekarang, mari kita implementasikan blok transformer. Blok ini akan menerima output dari lapisan embedding yang telah didefinisikan sebelumnya:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Sekarang, kita siap untuk mendefinisikan model transformer lengkap:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## Model Transformer BERT

**BERT** (Bidirectional Encoder Representations from Transformers) adalah jaringan transformer multi-layer yang sangat besar dengan 12 lapisan untuk *BERT-base*, dan 24 lapisan untuk *BERT-large*. Model ini pertama kali dilatih pada korpus teks yang sangat besar (WikiPedia + buku) menggunakan pelatihan tanpa pengawasan (memprediksi kata-kata yang disembunyikan dalam sebuah kalimat). Selama pelatihan awal, model ini menyerap pemahaman bahasa yang signifikan yang kemudian dapat dimanfaatkan dengan dataset lain melalui proses penyetelan ulang. Proses ini disebut **transfer learning**.

![gambar dari http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/id/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Ada banyak variasi arsitektur Transformer termasuk BERT, DistilBERT, BigBird, OpenGPT3, dan lainnya yang dapat disesuaikan lebih lanjut.

Mari kita lihat bagaimana kita dapat menggunakan model BERT yang telah dilatih sebelumnya untuk menyelesaikan masalah klasifikasi urutan tradisional kita. Kita akan meminjam ide dan beberapa kode dari [dokumentasi resmi](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Untuk memuat model yang telah dilatih sebelumnya, kita akan menggunakan **Tensorflow hub**. Pertama, mari kita muat vektorisasi khusus BERT:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Penting untuk menggunakan vectorizer yang sama seperti yang digunakan oleh jaringan asli saat dilatih. Selain itu, vectorizer BERT mengembalikan tiga komponen:
* `input_word_ids`, yaitu urutan nomor token untuk kalimat masukan
* `input_mask`, menunjukkan bagian mana dari urutan yang berisi masukan sebenarnya, dan bagian mana yang merupakan padding. Ini mirip dengan mask yang dihasilkan oleh lapisan `Masking`
* `input_type_ids` digunakan untuk tugas pemodelan bahasa, dan memungkinkan untuk menentukan dua kalimat masukan dalam satu urutan.

Kemudian, kita dapat membuat instans extractor fitur BERT:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Jadi, lapisan BERT mengembalikan sejumlah hasil yang berguna:
* `pooled_output` adalah hasil dari merata-ratakan semua token dalam urutan. Anda dapat menganggapnya sebagai embedding semantik cerdas dari seluruh jaringan. Ini setara dengan keluaran dari lapisan `GlobalAveragePooling1D` pada model kita sebelumnya.
* `sequence_output` adalah keluaran dari lapisan transformer terakhir (sesuai dengan keluaran `TransformerBlock` pada model kita di atas).
* `encoder_outputs` adalah keluaran dari semua lapisan transformer. Karena kita telah memuat model BERT dengan 4 lapisan (seperti yang mungkin Anda tebak dari namanya, yang mengandung `4_H`), model ini memiliki 4 tensor. Tensor terakhir sama dengan `sequence_output`.

Sekarang kita akan mendefinisikan model klasifikasi end-to-end. Kita akan menggunakan *definisi model fungsional*, di mana kita mendefinisikan input model, lalu memberikan serangkaian ekspresi untuk menghitung keluarannya. Kita juga akan membuat bobot model BERT tidak dapat dilatih, dan hanya melatih pengklasifikasi akhirnya:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Meskipun jumlah parameter yang dapat dilatih sedikit, prosesnya cukup lambat karena ekstraktor fitur BERT sangat berat secara komputasi. Tampaknya kita tidak berhasil mencapai akurasi yang memadai, baik karena kurangnya pelatihan maupun kurangnya parameter model.

Mari kita coba membuka kunci bobot BERT dan melatihnya juga. Ini membutuhkan tingkat pembelajaran yang sangat kecil, serta strategi pelatihan yang lebih hati-hati dengan **warmup**, menggunakan pengoptimal **AdamW**. Kita akan menggunakan paket `tf-models-official` untuk membuat pengoptimal:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Seperti yang Anda lihat, pelatihan berjalan cukup lambat - tetapi Anda mungkin ingin bereksperimen dan melatih model selama beberapa epoch (5-10) dan melihat apakah Anda bisa mendapatkan hasil terbaik dibandingkan dengan pendekatan yang telah kita gunakan sebelumnya.

## Perpustakaan Huggingface Transformers

Cara lain yang sangat umum (dan sedikit lebih sederhana) untuk menggunakan model Transformer adalah [paket HuggingFace](https://github.com/huggingface/), yang menyediakan blok bangunan sederhana untuk berbagai tugas NLP. Paket ini tersedia baik untuk Tensorflow maupun PyTorch, kerangka kerja jaringan saraf lain yang sangat populer.

> **Catatan**: Jika Anda tidak tertarik untuk melihat bagaimana perpustakaan Transformers bekerja - Anda dapat langsung melompat ke akhir notebook ini, karena Anda tidak akan melihat sesuatu yang secara substansial berbeda dari apa yang telah kita lakukan sebelumnya. Kita akan mengulangi langkah-langkah yang sama untuk melatih model BERT menggunakan perpustakaan yang berbeda dan model yang jauh lebih besar. Oleh karena itu, proses ini melibatkan pelatihan yang cukup lama, jadi Anda mungkin hanya ingin melihat kode saja.

Mari kita lihat bagaimana masalah kita dapat diselesaikan menggunakan [Huggingface Transformers](http://huggingface.co).


Hal pertama yang perlu kita lakukan adalah memilih model yang akan kita gunakan. Selain beberapa model bawaan, Huggingface memiliki [repositori model online](https://huggingface.co/models), di mana Anda dapat menemukan banyak model yang telah dilatih sebelumnya oleh komunitas. Semua model tersebut dapat dimuat dan digunakan hanya dengan menyediakan nama model. Semua file biner yang diperlukan untuk model tersebut akan otomatis diunduh.

Pada waktu tertentu, Anda mungkin perlu memuat model Anda sendiri. Dalam kasus ini, Anda dapat menentukan direktori yang berisi semua file terkait, termasuk parameter untuk tokenizer, file `config.json` dengan parameter model, bobot biner, dan sebagainya.

Dari nama model, kita dapat menginisialisasi baik model maupun tokenizer. Mari kita mulai dengan tokenizer:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Objek `tokenizer` berisi fungsi `encode` yang dapat langsung digunakan untuk mengenkripsi teks:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Kita juga dapat menggunakan tokenizer untuk mengenkripsi sebuah urutan dengan cara yang sesuai untuk diteruskan ke model, yaitu termasuk bidang `token_ids`, `input_mask`, dll. Kita juga dapat menentukan bahwa kita ingin tensor Tensorflow dengan memberikan argumen `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

Dalam kasus ini, kita akan menggunakan model BERT yang telah dilatih sebelumnya bernama `bert-base-uncased`. *Uncased* menunjukkan bahwa model ini tidak peka terhadap huruf besar dan kecil.

Saat melatih model, kita perlu memberikan urutan token yang telah di-tokenisasi sebagai input, dan oleh karena itu kita akan merancang alur pemrosesan data. Karena `tokenizer.encode` adalah fungsi Python, kita akan menggunakan pendekatan yang sama seperti pada unit sebelumnya dengan memanggilnya menggunakan `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Sekarang kita dapat memuat model sebenarnya menggunakan paket `BertForSequenceClassification`. Ini memastikan bahwa model kita sudah memiliki arsitektur yang diperlukan untuk klasifikasi, termasuk classifier akhir. Anda akan melihat pesan peringatan yang menyatakan bahwa bobot classifier akhir belum diinisialisasi, dan model memerlukan pra-pelatihan - itu sepenuhnya tidak masalah, karena itulah yang akan kita lakukan!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Seperti yang dapat Anda lihat dari `summary()`, model ini mengandung hampir 110 juta parameter! Kemungkinan, jika kita ingin tugas klasifikasi sederhana pada dataset yang relatif kecil, kita tidak ingin melatih lapisan dasar BERT:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Sekarang kita siap untuk memulai pelatihan!

> **Catatan**: Melatih model BERT skala penuh bisa sangat memakan waktu! Oleh karena itu, kita hanya akan melatihnya untuk 32 batch pertama. Ini hanya untuk menunjukkan bagaimana pelatihan model diatur. Jika Anda tertarik mencoba pelatihan skala penuh - cukup hapus parameter `steps_per_epoch` dan `validation_steps`, dan bersiaplah untuk menunggu!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Jika Anda meningkatkan jumlah iterasi dan menunggu cukup lama, serta melatih selama beberapa epoch, Anda dapat mengharapkan bahwa klasifikasi BERT memberikan akurasi terbaik! Hal ini karena BERT sudah cukup memahami struktur bahasa dengan baik, dan kita hanya perlu menyempurnakan pengklasifikasi akhirnya. Namun, karena BERT adalah model yang besar, seluruh proses pelatihan memakan waktu lama dan membutuhkan daya komputasi yang signifikan! (GPU, dan sebaiknya lebih dari satu).

> **Catatan:** Dalam contoh kita, kita menggunakan salah satu model BERT pra-latih terkecil. Ada model yang lebih besar yang kemungkinan memberikan hasil yang lebih baik.


## Poin Penting

Dalam unit ini, kita telah mempelajari arsitektur model terbaru yang berbasis **transformers**. Kita telah menerapkannya untuk tugas klasifikasi teks, namun model BERT juga dapat digunakan untuk ekstraksi entitas, menjawab pertanyaan, dan tugas NLP lainnya.

Model transformer mewakili teknologi terkini dalam NLP, dan dalam sebagian besar kasus, ini seharusnya menjadi solusi pertama yang Anda coba saat mengimplementasikan solusi NLP kustom. Namun, memahami prinsip dasar dari jaringan saraf berulang yang dibahas dalam modul ini sangatlah penting jika Anda ingin membangun model saraf yang lebih canggih.



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berusaha untuk memberikan hasil yang akurat, harap diingat bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang otoritatif. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
